In [12]:
# Importing dependencies
import wikipedia
import pandas as pd
from random import sample 
import time
from math import log10,sqrt

list_searches_nba = ['Los Angeles Lakers', 'Golden State Warriors', 'Toronto Raptors',
                'Milwaukee Bucks', 'LA Clippers', 'Boston Celtics', 'Denver Nuggets',
                'Utah Jazz', 'Miami Heat', 'Houston Rockets', 'Dallas Mavericks',
                    'Atlanta Hawks', 'Brooklyn Nets', 'Charlotte Hornets', 'Chicago Bulls',
                    'Cleveland Cavaliers', 'Detroit Pistons', 'Indiana Pacers', 'Memphis Grizzlies',
                    'Minnesota Timberwolves', 'New Orleans Pelicans', 'New York Knicks', 'Oklahoma City Thunder']
list_searches_cl = ['Manchester City', 'Paris Saint Germain', 'Real Madrid', 'FC Barcelona',
                    'Liverpool', 'Juventus', 'Valancia', 'Atalanta Bergamo',
                    'FC Bayern Munchen', 'Atlético Madrid', 'SSC Napoli', 'Borussia Dortmund',
                    'Olympique Lyon', 'Tottenham Hotspur', 'RasenBallsport Leipzig', 'Ajax Amsterdam',
                    'Dinamo Zagreb', 'APOEL Nicosia', 'Dynamo kiev', 'Bayer 04 Leverkusen', 'Internazionale',
                    'FC Red Bull Salzburg', 'Galatasaray', 'Olympiacos', 'Hoffenhheim', 'Schalke 04',
                    'Feyenoord', 'PSV']

# Function for extracting the wikipedia pages of teams
def import_wikipedia(list_searches):
    data_searches = []
    for i in list_searches:
        p = wikipedia.page(i)
        data_searches.append(p.content)
    return data_searches
data_nba = import_wikipedia(list_searches_nba)
data_cl = import_wikipedia(list_searches_cl)
datasets = data_nba + data_cl

lst_search = list_searches_nba + list_searches_cl
stopwords= ['a','able','about','across','after','all','almost','also','am','among','an','and','any','are','as','at','be','because','been','but','by',
            'can','cannot','could','dear','did','do','does','either','else','ever','every','for','from','get','got','had','has','have','he','her','hers',
            'him','his','how','however','i','if','in','into','is','it','its','just','least','let','like','likely','may','me','might','most','must','my',
            'neither','no','nor','not','of','off','often','on','only','or','other','our','own','rather','said','say','says','she','should','since','so',
            'some','than','that','the','their','them','then','there','these','they','this','tis','to','too','twas','us','wants','was','we','were','what',
            'when','where','which','while','who','whom','why','will','with','would','yet','you','your']

In [13]:
def mapper1(data, lst_search):
    final_output = []
    for i in range(len(data)):
        output_not_sorted = []
        filename = lst_search[i]
        # remove whitespace
        line = data[i].strip()
        # split into words
        words = line.split()
        for word in words:
            word=word.lower()
            if word not in stopwords:
                z=word+' * '+filename
                output = '%s \t %s' % (z, 1)
                output_not_sorted.append(output)
        sorted_output = sorted(output_not_sorted)
        final_output.append(sorted_output)

    return final_output

In [14]:
def reducer1(map_result):
    current_word = None
    current_count = 0
    word = None
    final_output = []

    for j in map_result:
        for line in j:
            line = line.strip()
            word, count = line.split('\t', 1)

            try:
                count = int(count)
            except ValueError:
                continue
            
            # The if-switch works because the output is mapped by key (here: word),
            # this is how it also works in Hadoop
            if current_word == word:
                current_count += count
            else:
                if current_word:
                    output = '%s \t %s' % (current_word, current_count)
                    final_output.append(output)
                current_count = count
                current_word = word

    # Outputting the last word if needed
    if current_word == word:
        output = '%s \t %s' % (current_word, current_count)
        final_output.append(output)
    return final_output

In [15]:
%%time
map_result1 = mapper1(datasets, lst_search)
reduce_result1 = reducer1(map_result1)

CPU times: user 1.08 s, sys: 21.9 ms, total: 1.1 s
Wall time: 1.11 s


In [16]:
def mapper2(data_reducer):
    final_output = []
    for line in data_reducer:
        # remove leading and trailing whitespace
        line = line.strip()
        # split the line into words
        wordfilename,count=line.split('\t',1)
        wordfilename = wordfilename.strip()
        try:
            word,filename=wordfilename.split('*')
        except:
            pass
        z=word+' '+count;
        output = '%s \t %s' % (filename, z)
        final_output.append(output)
    return final_output

In [17]:
def reducer2(map_result):
    current_word = None
    prev_filename = None
    current_count = 0
    word = None
    N=0
    df={}
    l1=[]
    final_output = []
    
    for line in map_result:
        line = line.strip()
        l1.append(line)
        filename,wordcount = line.split('\t', 1)
        wordcount = wordcount.strip()
        word,count = wordcount.split(' ', 1)
        count=int(count)
        if prev_filename == filename:
            N+=count
        else:
            if prev_filename != None:
                df[prev_filename]=N
            N=0
            prev_filename = filename
    df[prev_filename]=N

    for h in l1:
        filename,wordcount = h.split('\t', 1)
        wordcount = wordcount.strip()
        word,count = wordcount.split(' ', 1) 
        for k in df:
            if filename == k:
                wf=word+' '+filename
                nN=count+' '+str(df[k])
                output = '%s \t %s' % (wf,nN)
                final_output.append(output)
    return final_output

In [18]:
%%time
map_result2 = mapper2(reduce_result1)
reduce_result2 = reducer2(map_result2)

CPU times: user 751 ms, sys: 23.1 ms, total: 774 ms
Wall time: 781 ms


In [19]:
def mapper3(reduce_result):
    output_list = []
    for line in reduce_result:
        line = line.strip()
        wf,nN=line.split('\t',1)
        w,f=wf.split(' ',1)
        z=f+' * '+nN+' '+str(1)
        output = '%s \t %s' % (w,z)
        output_list.append(output)
    final_output = sorted(output_list)
    return final_output

In [20]:
def reducer3(map_result):
    prev_word = None
    count = 1 
    word = None
    final_output = []
    for line in map_result:
        line = line.strip()
        w,z= line.split('\t', 1)
        w = w.strip()
        z = z.strip()
        f,nNc = z.split('*',1)
        f = f.strip()
        nNc = nNc.strip()
        n,Nc=nNc.split(' ',1)
        N,c=Nc.split(' ',1)
        if prev_word == w:
            count += int(c)
        else:
            if prev_word != None:
                output=w+' '+f+' \t '+n+' '+N+' '+str(count)
                final_output.append(output)
            count=1
            prev_word = w
    return final_output

In [21]:
%%time
map_result3 = mapper3(reduce_result2)
# reduce_result3 = reducer3(map_result3)

CPU times: user 216 ms, sys: 7.33 ms, total: 223 ms
Wall time: 224 ms


In [22]:
%%time
reduce_result3 = reducer3(map_result3)

CPU times: user 249 ms, sys: 5.73 ms, total: 255 ms
Wall time: 254 ms


In [23]:
def mapper4(reduce_result):
    D=51.0
    final_output = []
    # input comes from STDIN (standard input)
    for line in reduce_result:
        # remove leading and trailing whitespace
        line = line.strip()
        # split the line into words
        wf,nNm=line.split('\t',1)
        wf = wf.strip()
        nNm = nNm.strip()
        n,N,m=nNm.split(' ',2)
        n=float(n)
        N=float(N)
        m=float(m)
        tfidf= (n/N)*log10(D/m)
        output = '%s \t %s' % (wf,tfidf)
        final_output.append(output)
    return final_output

In [24]:
%%time
final = mapper4(reduce_result3)

CPU times: user 134 ms, sys: 5.46 ms, total: 140 ms
Wall time: 139 ms


In [25]:
def compute_computation_times(data_nba, data_cl):
    running_times = []
    for i in range(5, 51, 5):
        new_data = [sample(data_nba,i), sample(data_cl,i)]
        data1, data2 = compute_frequency_matrix(new_data)
        start_time = time.time()
        igm1, igm2 = compute_igm(data1, data2)
        running_times.append(time.time() - start_time)
    return running_times

In [26]:
# %%time
running_times = []
for i in range(5, 51, 5):
    new_data = sample(datasets,i)
    lst_searches = sample(lst_search, i)
    start_time = time.time()
    map_result1 = mapper1(new_data, lst_searches)
    reduce_result1 = reducer1(map_result1)
    map_result2 = mapper2(reduce_result1)
    reduce_result2 = reducer2(map_result2)
    map_result3 = mapper3(reduce_result2)
    reduce_result3 = reducer3(map_result3)
    final = mapper4(reduce_result3)
    running_times.append(time.time() - start_time)

In [19]:
running_times

[0.24024295806884766,
 0.48619604110717773,
 0.6151602268218994,
 0.8494110107421875,
 1.1304759979248047,
 1.2444140911102295,
 1.5537216663360596,
 1.8164231777191162,
 2.1589066982269287,
 2.3509411811828613]

In [42]:
final_output = []
final_tfidf = []
D=51.0
# input comes from STDIN (standard input)
for line in reduce_result3:
    # remove leading and trailing whitespace
    line = line.strip()
    # split the line into words
    wf,nNm=line.split('\t',1)
    wf = wf.strip()
    w,f = wf.split(' ',1)
    nNm = nNm.strip()
    n,N,m=nNm.split(' ',2)
    n=float(n)
    N=float(N)
    m=float(m)
    tfidf= (n/N)*log10(D/m)
    output = w
    final_output.append(output)
    final_tfidf.append(tfidf)

In [43]:
data_tuples = list(zip(final_output,final_tfidf))
df1 = pd.DataFrame(data_tuples, columns=['word','tfidf'])
df1.sort_values('tfidf', ascending=False).head(15)

,word,tfidf
16676,hawks,0.056722
22034,napoli,0.049946
25048,psp,0.046178
13160,dynamo,0.043161
15529,galatasaray,0.042655
7033,apoel,0.040384
27694,serie,0.036917
22902,olympiacos,0.032282
31743,valancia,0.031538
1602,(b.,0.027541


In [ ]:
# Local TF-IDF
[2.8284547328948975,
 11.59837794303894,
 12.069886922836304,
 17.457234859466553,
 28.922400951385498,
 35.147900342941284,
 44.212185859680176,
 50.5285918712616,
 57.81522607803345,
 64.94160890579224]

In [ ]:
# Parallel TF-IDF
[2.1748712062835693,
 3.716153621673584,
 9.97753381729126,
 14.972901582717896,
 14.40234088897705,
 24.855359315872192,
 28.49886393547058,
 30.41658616065979,
 39.89217281341553,
 45.6888325214386]

In [ ]:
# [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
# kijken nog naar hoeveel bytes het is

In [14]:
len(map_result3)

99358

In [15]:
len(reduce_result3)

34123

In [ ]:
# Old Reducer
# Gaat nog wat mis bij eerste prev_word = None en op het einde duurt die lang

def reducer3(map_result):
    prev_word = None
    count = 1 
    word = None
    df={}
    l1=[]
    final_output = []
    # input comes from STDIN
    for line in map_result:
        line = line.strip()
        w,z= line.split('\t', 1)
        w = w.strip()
        z = z.strip()
        f,nNc = z.split('*',1)
        f = f.strip()
        nNc = nNc.strip()
        n,Nc=nNc.split(' ',1)
        N,c=Nc.split(' ',1)
        if prev_word == w:
            # count = count+int(c)
            count += int(c)
        else:
            if prev_word != None:
                q=n+' '+N+' '+str(count)
                df[prev_word]=q
                j=prev_word+' '+f
                l1.append(j)
            count=1
            prev_word = w


#     q=n+' '+N+' '+str(count)
#     df[prev_word]=q
#     j=prev_word+' '+f
#     l1.append(j)

    for h in l1:
        w,f=h.split(' ',1)
        w = w.strip()
        f = f.strip()
        for d in df:
            if w == d:
                output = '%s \t %s' % (h,df[d])
                final_output.append(output)
    return final_output